In [130]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import RGCNConv, Node2Vec
from torch_geometric.utils import negative_sampling
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

In [131]:
# pip install torch-geometric -f https://data.pyg.org/whl/torch-2.5.1+cpu.html

In [132]:
# pip install pyg-lib torch-scatter torch-sparse torch-cluster -f https://data.pyg.org/whl/torch-2.5.1+cpu.html


In [133]:
# pip install torch==2.5.1 torchvision==0.20.1 torchaudio==2.5.1 --index-url https://download.pytorch.org/whl/cpu

In [134]:
from owlready2 import get_ontology
import pandas as pd

# Step 1: Load the ontology
ontology_path = "enriched_ppio_ontology_final_54_small.owl"  # Replace with your OWL file path
ontology = get_ontology(ontology_path).load()

# Step 2: Extract triples for patients with disease statuses and their relationships
def extract_patient_triples(ontology, target_statuses, relations_of_interest):
    """
    Extract triples (subject, predicate, object) for patients with specific disease statuses
    and related properties like hasSample, isAssociatedWithPathway, hasGeneExpression.
    """
    triples = []

    # Iterate through all individuals in the ontology
    for patient in ontology.individuals():
        if "Patient" in [cls.name for cls in patient.is_a]:  # Filter only Patient instances
            disease_status = None
            samples = []

            # Extract properties of the patient
            for prop in patient.get_properties():
                for value in prop[patient]:
                    if prop.python_name == "hasDiseaseStatus":
                        disease_status = value
                    elif prop.python_name == "hasSample":
                        samples.append(value.name)

            # If the patient matches the target disease status
            if disease_status in target_statuses:
                triples.append((patient.name, "hasDiseaseStatus", disease_status))

                # Extract relationships for associated samples
                for sample_name in samples:
                    triples.append((patient.name, "hasSample", sample_name))
                    # Find relationships for the sample
                    for sample in ontology.individuals():
                        if sample.name == sample_name:
                            for prop in sample.get_properties():
                                if prop.python_name in relations_of_interest:
                                    for value in prop[sample]:
                                        if hasattr(value, 'name'):
                                            triples.append((sample.name, prop.python_name, value.name))

    return triples

# Step 3: Define disease statuses and relationships of interest
target_statuses = ["sepsis survivor", "sepsis non-survivor"]
relations_of_interest = ["isAssociatedWithPathway", "hasGeneExpression"]

# Step 4: Extract triples
triples = extract_patient_triples(ontology, target_statuses, relations_of_interest)

# Step 5: Convert to DataFrame and remove duplicates
triples_df1 = pd.DataFrame(triples, columns=["Subject", "Predicate", "Object"]).drop_duplicates()

# Step 6: Display the results
print("\n### Final Triples DataFrame ###")
print(triples_df1.head(20))



### Final Triples DataFrame ###
              Subject          Predicate             Object
0        Patient_S_29   hasDiseaseStatus    sepsis survivor
1        Patient_S_29          hasSample  Sample_GSM1317945
2   Sample_GSM1317945  hasGeneExpression         Gene_60496
3   Sample_GSM1317945  hasGeneExpression            Gene_18
4   Sample_GSM1317945  hasGeneExpression         Gene_10347
5   Sample_GSM1317945  hasGeneExpression           Gene_215
6   Sample_GSM1317945  hasGeneExpression            Gene_23
7   Sample_GSM1317945  hasGeneExpression            Gene_26
8   Sample_GSM1317945  hasGeneExpression         Gene_27034
9   Sample_GSM1317945  hasGeneExpression            Gene_37
10  Sample_GSM1317945  hasGeneExpression         Gene_91452
11  Sample_GSM1317945  hasGeneExpression         Gene_22985
12  Sample_GSM1317945  hasGeneExpression            Gene_51
13  Sample_GSM1317945  hasGeneExpression          Gene_2180
14  Sample_GSM1317945  hasGeneExpression         Gene_84532
15  Sam

In [135]:

# Step 2: Extract relevant triples
def extract_gene_triples(ontology, relations_of_interest):
    """
    Extract triples (subject, predicate, object) for genes with specific relationships.
    """
    triples = []

    # Iterate through individuals in the ontology
    for entity in ontology.individuals():
        # Process only Gene entities
        if "Gene" in [cls.name for cls in entity.is_a]:
            for prop in entity.get_properties():
                # Match properties of interest
                if prop.python_name in relations_of_interest:
                    for value in prop[entity]:
                        if hasattr(value, 'name'):  # Check if value is another entity
                            triples.append((entity.name, prop.python_name, value.name))
    return triples

# Step 3: Define relationships of interest
relations_of_interest = [
 "isAssociatedWithPathway"
]

# Step 4: Extract triples
gene_triples = extract_gene_triples(ontology, relations_of_interest)

# Step 5: Convert to DataFrame
triples_df2 = pd.DataFrame(gene_triples, columns=["Subject", "Predicate", "Object"])

# Step 6: Display the triples
print("\n### Extracted Gene Triples ###")
print(triples_df2.head(20))  # Display the first 20 rows




### Extracted Gene Triples ###
       Subject                Predicate                 Object
0   Gene_60496  isAssociatedWithPathway        Pathway_0000000
1   Gene_60496  isAssociatedWithPathway  Pathway_R-HSA-1430728
2   Gene_60496  isAssociatedWithPathway   Pathway_R-HSA-196849
3   Gene_60496  isAssociatedWithPathway   Pathway_R-HSA-196854
4      Gene_18  isAssociatedWithPathway        Pathway_0000000
5      Gene_18  isAssociatedWithPathway   Pathway_R-HSA-112315
6      Gene_18  isAssociatedWithPathway   Pathway_R-HSA-112316
7   Gene_10347  isAssociatedWithPathway        Pathway_0000000
8   Gene_10347  isAssociatedWithPathway   Pathway_R-HSA-382551
9     Gene_215  isAssociatedWithPathway        Pathway_0000000
10    Gene_215  isAssociatedWithPathway  Pathway_R-HSA-1643685
11    Gene_215  isAssociatedWithPathway  Pathway_R-HSA-1430728
12    Gene_215  isAssociatedWithPathway   Pathway_R-HSA-556833
13    Gene_215  isAssociatedWithPathway   Pathway_R-HSA-382551
14    Gene_215  isAssoc

In [136]:
# Step 1: Concatenate the two DataFrames
combined_df = pd.concat([triples_df1, triples_df2], ignore_index=True)

# Step 2: Display the combined DataFrame
print("\n### Combined Triples DataFrame ###")
print(combined_df)


### Combined Triples DataFrame ###
                  Subject                Predicate                Object
0            Patient_S_29         hasDiseaseStatus       sepsis survivor
1            Patient_S_29                hasSample     Sample_GSM1317945
2       Sample_GSM1317945        hasGeneExpression            Gene_60496
3       Sample_GSM1317945        hasGeneExpression               Gene_18
4       Sample_GSM1317945        hasGeneExpression            Gene_10347
...                   ...                      ...                   ...
144851        Gene_170960  isAssociatedWithPathway  Pathway_R-HSA-212436
144852        Gene_155061  isAssociatedWithPathway       Pathway_0000000
144853        Gene_155061  isAssociatedWithPathway   Pathway_R-HSA-74160
144854        Gene_155061  isAssociatedWithPathway   Pathway_R-HSA-73857
144855        Gene_155061  isAssociatedWithPathway  Pathway_R-HSA-212436

[144856 rows x 3 columns]


In [101]:
# Step 2: Load DataFrame and Preprocess
# Assume combined_df_stand is already loaded
entities = list(set(combined_df_stand['Subject']).union(set(combined_df_stand['Object'])))
relations = list(set(combined_df_stand['Predicate']))

# Create mappings
entity_to_id = {entity: idx for idx, entity in enumerate(entities)}
relation_to_id = {relation: idx for idx, relation in enumerate(relations)}

# Map entities and relations to IDs
combined_df_stand['Subject_ID'] = combined_df_stand['Subject'].map(entity_to_id)
combined_df_stand['Object_ID'] = combined_df_stand['Object'].map(entity_to_id)
combined_df_stand['Predicate_ID'] = combined_df_stand['Predicate'].map(relation_to_id)

# Prepare edges and types
edge_index = torch.tensor(combined_df_stand[['Subject_ID', 'Object_ID']].values.T, dtype=torch.long)
edge_type = torch.tensor(combined_df_stand['Predicate_ID'].values, dtype=torch.long)


In [102]:
from sklearn.model_selection import train_test_split

# Step 3: Split Data into Train, Validation, and Test
train_df, temp_df = train_test_split(combined_df_stand, test_size=0.3, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)




In [103]:
def prepare_edges(df):
    edge_index = torch.tensor(df[['Subject_ID', 'Object_ID']].values.T, dtype=torch.long)
    edge_type = torch.tensor(df['Predicate_ID'].values, dtype=torch.long)
    return edge_index, edge_type

train_edge_index, train_edge_type = prepare_edges(train_df)
val_edge_index, val_edge_type = prepare_edges(val_df)
test_edge_index, test_edge_type = prepare_edges(test_df)

In [104]:
# pip install torch torch-geometric torch-scatter torch-sparse torch-cluster pyg-lib

In [105]:
# pip install torch==2.5.1+cpu torchvision==0.15.2+cpu -f https://download.pytorch.org/whl/cpu

In [106]:
import torch
import torch_geometric
from torch_geometric.nn import Node2Vec

print("PyTorch Geometric version:", torch_geometric.__version__)
print("Torch version:", torch.__version__)


PyTorch Geometric version: 2.6.1
Torch version: 2.5.1


In [107]:
import torch
import torch_geometric
from torch_geometric.nn import Node2Vec

print("PyTorch Geometric version:", torch_geometric.__version__)
print("Torch version:", torch.__version__)


PyTorch Geometric version: 2.6.1
Torch version: 2.5.1


In [108]:
pip show pyg-lib


Name: pyg_lib
Version: 0.4.0+pt25
Summary: Low-Level Graph Neural Network Operators for PyG
Home-page: https://github.com/pyg-team/pyg-lib
Author: PyG Team
Author-email: team@pyg.org
License: 
Location: /opt/homebrew/Cellar/jupyterlab/4.3.1_1/libexec/lib/python3.12/site-packages
Requires: 
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [109]:
pip show torch-cluster

Name: torch_cluster
Version: 1.6.3
Summary: PyTorch Extension Library of Optimized Graph Cluster Algorithms
Home-page: https://github.com/rusty1s/pytorch_cluster
Author: Matthias Fey
Author-email: matthias.fey@tu-dortmund.de
License: 
Location: /opt/homebrew/Cellar/jupyterlab/4.3.1_1/libexec/lib/python3.12/site-packages
Requires: scipy
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [110]:
# Step 4: Initialize Node Features Using Node2Vec
num_entities = len(entity_to_id)
node2vec = Node2Vec(edge_index, embedding_dim=128, walk_length=10, context_size=5, walks_per_node=5)
node2vec.train()
node_features = node2vec.embedding.weight.data
node_features = (node_features - node_features.mean(dim=0)) / (node_features.std(dim=0) + 1e-6)
node_features = node_features / torch.max(torch.abs(node_features))  # Normalize to [-1, 1]



In [111]:
# Step 1: Define R-GCN Model with Relation Embeddings and MLP Scoring
class RGCNModel(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, num_relations):
        super(RGCNModel, self).__init__()
        self.conv1 = RGCNConv(input_dim, hidden_dim, num_relations=num_relations)
        self.conv2 = RGCNConv(hidden_dim, output_dim, num_relations=num_relations)
        self.scoring = torch.nn.Linear(2 * output_dim, 1)  # Scoring via MLP

    def forward(self, x, edge_index, edge_type):
        x = self.conv1(x, edge_index, edge_type)
        x = F.relu(x)
        x = self.conv2(x, edge_index, edge_type)
        return x

    def predict(self, src, dst):
        return self.scoring(torch.cat([src, dst], dim=-1))


In [112]:
# Step 2: Compute ranking metrics
def compute_ranking_metrics(embeddings, pos_edge_index):
    ranks = []
    hits_at_1, hits_at_3, hits_at_10 = 0, 0, 0

    for i in range(pos_edge_index.size(1)):
        src, dst = pos_edge_index[0, i], pos_edge_index[1, i]

        # Compute similarity scores for all nodes with respect to the source node
        scores = torch.matmul(embeddings[src], embeddings.t())
        sorted_indices = torch.argsort(scores, descending=True)

        # Find the rank of the positive target node
        rank = (sorted_indices == dst).nonzero(as_tuple=True)[0].item() + 1
        ranks.append(rank)

        # Update Hits@K counts
        if rank <= 1: hits_at_1 += 1
        if rank <= 3: hits_at_3 += 1
        if rank <= 10: hits_at_10 += 1

    # Compute Mean Rank and MRR
    mean_rank = np.mean(ranks)
    mrr = np.mean([1.0 / rank for rank in ranks])

    # Normalize Hits@K
    total_edges = len(ranks)
    hits_at_1 /= total_edges
    hits_at_3 /= total_edges
    hits_at_10 /= total_edges

    return mean_rank, mrr, hits_at_1, hits_at_3, hits_at_10

def compute_detailed_ranking_metrics(embeddings, pos_edge_index):
    ranks = {"overall": [], "head": [], "tail": []}
    hits_at_k = {"overall": [0, 0, 0], "head": [0, 0, 0], "tail": [0, 0, 0]}

    for i in range(pos_edge_index.size(1)):
        src, dst = pos_edge_index[0, i], pos_edge_index[1, i]

        # Compute similarity scores for predicting tail
        tail_scores = torch.matmul(embeddings[src], embeddings.t())
        sorted_tail_indices = torch.argsort(tail_scores, descending=True)
        tail_rank = (sorted_tail_indices == dst).nonzero(as_tuple=True)[0].item() + 1
        ranks["overall"].append(tail_rank)
        ranks["tail"].append(tail_rank)

        # Compute similarity scores for predicting head
        head_scores = torch.matmul(embeddings[dst], embeddings.t())
        sorted_head_indices = torch.argsort(head_scores, descending=True)
        head_rank = (sorted_head_indices == src).nonzero(as_tuple=True)[0].item() + 1
        ranks["overall"].append(head_rank)
        ranks["head"].append(head_rank)

    for key in ranks.keys():
        total_ranks = len(ranks[key])
        hits_at_k[key][0] = sum(1 for rank in ranks[key] if rank <= 1) / total_ranks
        hits_at_k[key][1] = sum(1 for rank in ranks[key] if rank <= 3) / total_ranks
        hits_at_k[key][2] = sum(1 for rank in ranks[key] if rank <= 10) / total_ranks
        ranks[key] = {
            "mean_rank": np.mean(ranks[key]),
            "mrr": np.mean([1.0 / rank for rank in ranks[key]]),
            "hits_at_1": hits_at_k[key][0],
            "hits_at_3": hits_at_k[key][1],
            "hits_at_10": hits_at_k[key][2],
        }

    return ranks

In [113]:
def safe_negative_sampling(edge_index, num_nodes, num_neg_samples):
    return negative_sampling(
        edge_index=edge_index,
        num_nodes=num_nodes,
        num_neg_samples=num_neg_samples,
    )

In [114]:
# Define loss function for training
def compute_loss(embeddings, edge_index, edge_type):
    # Using binary cross-entropy loss with negative sampling
    negative_edge_index = safe_negative_sampling(
        edge_index=edge_index,
        num_nodes=embeddings.size(0),
        num_neg_samples=edge_index.size(1),
    )
    positive_scores = torch.sum(embeddings[edge_index[0]] * embeddings[edge_index[1]], dim=1)
    negative_scores = torch.sum(embeddings[negative_edge_index[0]] * embeddings[negative_edge_index[1]], dim=1)

     # Debug scores
    print("Positive Scores Stats:", positive_scores.min().item(), positive_scores.max().item())
    print("Negative Scores Stats:", negative_scores.min().item(), negative_scores.max().item())

    loss = -torch.log(torch.sigmoid(positive_scores)).mean() - torch.log(torch.sigmoid(-negative_scores)).mean()
    return loss

In [115]:
# Step 6: Train the RGCN Model
model = RGCNModel(input_dim=128, hidden_dim=64, output_dim=128, num_relations=len(relation_to_id))
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)



In [116]:
# Check node features
assert not torch.isnan(node_features).any(), "Node features contain NaN values!"
assert not torch.isinf(node_features).any(), "Node features contain Inf values!"

# Check edge index
assert not torch.isnan(edge_index).any(), "Edge index contains NaN values!"
assert not torch.isinf(edge_index).any(), "Edge index contains Inf values!"

# Check edge type
assert not torch.isnan(edge_type).any(), "Edge type contains NaN values!"
assert not torch.isinf(edge_type).any(), "Edge type contains Inf values!"

# Print statistics for debugging
print("Node Features Stats:", node_features.min().item(), node_features.max().item())
print("Edge Index Shape:", edge_index.shape)
print("Edge Type Shape:", edge_type.shape)


Node Features Stats: -1.0 0.9520599246025085
Edge Index Shape: torch.Size([2, 239460])
Edge Type Shape: torch.Size([239460])


In [117]:
assert not torch.isinf(node_features).any(), "Node features contain Inf values!"


In [118]:
assert not torch.isnan(edge_index).any(), "Edge index contains NaN values!"


In [119]:
assert not torch.isinf(edge_index).any(), "Edge index contains Inf values!"

In [120]:
def train(model, optimizer, node_features, train_edge_index, train_edge_type, val_edge_index, val_edge_type, num_epochs=50):
    model.train()
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        
        # Forward pass
        embeddings = model(node_features, train_edge_index, train_edge_type)
        
        # Compute loss
        loss = compute_loss(embeddings, train_edge_index, train_edge_type)
        
        # Backward pass
        loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        # Update weights
        optimizer.step()
        
        # Validation metrics
        model.eval()
        with torch.no_grad():
            val_embeddings = model(node_features, val_edge_index, val_edge_type)
            mean_rank, mrr, hits_at_1, hits_at_3, hits_at_10 = compute_ranking_metrics(val_embeddings, val_edge_index)
        
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}, Val MRR: {mrr:.4f}")

train(model, optimizer, node_features, train_edge_index, train_edge_type, val_edge_index, val_edge_type)


Positive Scores Stats: -1.4832713603973389 7.991677284240723
Negative Scores Stats: -1.453662633895874 12.365680694580078
Epoch 1/50, Loss: 2.6425, Val MRR: 0.0041
Positive Scores Stats: -1.508592963218689 7.899600982666016
Negative Scores Stats: -1.154632568359375 12.025553703308105
Epoch 2/50, Loss: 2.5929, Val MRR: 0.0041
Positive Scores Stats: -1.5330815315246582 7.810639381408691
Negative Scores Stats: -1.411097526550293 11.835808753967285
Epoch 3/50, Loss: 2.5374, Val MRR: 0.0040
Positive Scores Stats: -1.5565295219421387 7.7225022315979
Negative Scores Stats: -1.4021825790405273 12.959908485412598
Epoch 4/50, Loss: 2.4920, Val MRR: 0.0040
Positive Scores Stats: -1.57887864112854 7.635150909423828
Negative Scores Stats: -1.6067343950271606 13.703432083129883
Epoch 5/50, Loss: 2.4406, Val MRR: 0.0040
Positive Scores Stats: -1.6003221273422241 7.549386024475098
Negative Scores Stats: -1.3479890823364258 12.467480659484863
Epoch 6/50, Loss: 2.3963, Val MRR: 0.0039
Positive Scores St

In [121]:
# Step 6: Evaluate on Validation or Test Set
print("Validation Metrics:")
model.eval()
with torch.no_grad():
    val_embeddings = model(node_features, edge_index, edge_type)
    mean_rank, mrr, hits_at_1, hits_at_3, hits_at_10 = compute_ranking_metrics(val_embeddings, edge_index)
    ranking_metrics = compute_detailed_ranking_metrics(val_embeddings, edge_index)
    print(f"Mean Rank (MR): {mean_rank:.4f}")
    print(f"Mean Reciprocal Rank (MRR): {mrr:.4f}")
    print(f"Hits@1: {hits_at_1:.4f}")
    print(f"Hits@3: {hits_at_3:.4f}")
    print(f"Hits@10: {hits_at_10:.4f}")


Validation Metrics:
Mean Rank (MR): 1005.6535
Mean Reciprocal Rank (MRR): 0.0022
Hits@1: 0.0000
Hits@3: 0.0002
Hits@10: 0.0010


In [122]:
# Create DataFrame for detailed metrics
performance_data_RGCN = {
    "Metric": ["Mean Rank", "Mean Reciprocal Rank (MRR)", "Hits@1", "Hits@3", "Hits@10"],
    "Overall": [
        ranking_metrics["overall"]["mean_rank"],
        ranking_metrics["overall"]["mrr"],
        ranking_metrics["overall"]["hits_at_1"],
        ranking_metrics["overall"]["hits_at_3"],
        ranking_metrics["overall"]["hits_at_10"],
    ],
    "Head Prediction": [
        ranking_metrics["head"]["mean_rank"],
        ranking_metrics["head"]["mrr"],
        ranking_metrics["head"]["hits_at_1"],
        ranking_metrics["head"]["hits_at_3"],
        ranking_metrics["head"]["hits_at_10"],
    ],
    "Tail Prediction": [
        ranking_metrics["tail"]["mean_rank"],
        ranking_metrics["tail"]["mrr"],
        ranking_metrics["tail"]["hits_at_1"],
        ranking_metrics["tail"]["hits_at_3"],
        ranking_metrics["tail"]["hits_at_10"],
    ],
}

metrics_df = pd.DataFrame(performance_data_RGCN)
print(metrics_df)

                       Metric     Overall  Head Prediction  Tail Prediction
0                   Mean Rank  950.816126       895.978740      1005.653512
1  Mean Reciprocal Rank (MRR)    0.003170         0.004129         0.002211
2                      Hits@1    0.000025         0.000025         0.000025
3                      Hits@3    0.000651         0.001140         0.000163
4                     Hits@10    0.003339         0.005725         0.000952


In [123]:
# Step 7: Testing (Same as validation)
print("Testing Metrics:")
with torch.no_grad():
    test_embeddings = model(node_features, edge_index, edge_type)
    mean_rank, mrr, hits_at_1, hits_at_3, hits_at_10 = compute_ranking_metrics(test_embeddings, edge_index)
    ranking_metrics = compute_detailed_ranking_metrics(test_embeddings, edge_index)
    print(f"Mean Rank (MR): {mean_rank:.4f}")
    print(f"Mean Reciprocal Rank (MRR): {mrr:.4f}")
    print(f"Hits@1: {hits_at_1:.4f}")
    print(f"Hits@3: {hits_at_3:.4f}")
    print(f"Hits@10: {hits_at_10:.4f}")

Testing Metrics:
Mean Rank (MR): 1005.6535
Mean Reciprocal Rank (MRR): 0.0022
Hits@1: 0.0000
Hits@3: 0.0002
Hits@10: 0.0010


In [162]:
from owlready2 import get_ontology

# Load the ontology
ontology_path = "enriched_ppio_ontology_final_54_small.owl"  # Replace with your local file path
ontology = get_ontology(ontology_path).load()

# Extract class names
class_names = [cls.name for cls in ontology.classes()]

# Extract relationships (object properties)
object_properties = [prop.name for prop in ontology.object_properties()]

# Extract annotation properties (metadata fields)
annotation_properties = [prop.name for prop in ontology.annotation_properties()]

# Display extracted information
ontology_summary = {
    "Number of Classes": len(class_names),
    "Number of Object Properties": len(object_properties),
    "Number of Annotation Properties": len(annotation_properties),
    "Sample Classes": class_names[:10],  # Show only first 10 classes
    "Sample Object Properties": object_properties[:10],
    "Sample Annotation Properties": annotation_properties[:10],
}

print(ontology_summary)


{'Number of Classes': 51573, 'Number of Object Properties': 21, 'Number of Annotation Properties': 79, 'Sample Classes': ['GO_0003674', 'GO_0005575', 'GO_0008150', 'GO_0000001', 'GO_0048308', 'GO_0048311', 'GO_0000002', 'GO_0007005', 'GO_0000003', 'GO_0000004'], 'Sample Object Properties': ['BFO_0000050', 'BFO_0000051', 'BFO_0000066', 'RO_0002091', 'RO_0002092', 'RO_0002093', 'RO_0002211', 'RO_0002212', 'RO_0002213', 'isAssociatedWithPathway'], 'Sample Annotation Properties': ['comment', 'label', 'deprecated', 'description', 'title', 'default-namespace', 'hasOBOFormatVersion', 'hasDbXref', 'hasOBONamespace', 'id']}


## Extract Nodes, Edges, and Metadata

In [166]:
# from owlready2 import get_ontology
# import pandas as pd
# import torch
# from torch_geometric.data import Data
# from torch_geometric.utils import negative_sampling
# from torch_geometric.nn import GCNConv

# # Load the ontology
# ontology_path = "enriched_ppio_ontology_final_54_small.owl"  # Path to your OWL file
# ontology = get_ontology(ontology_path).load()

# # Define relationships to extract
# relations_of_interest = ["hasSample", "hasGeneExpression", "isAssociatedWithPathway", "isAssociatedWith"]

# # Initialize node and edge storage
# nodes = set()
# edges = []
# metadata = {}

# # Extract nodes and edges
# for entity in ontology.individuals():
#     # Add entity to nodes
#     nodes.add(entity.name)
    
#     # Extract relationships for the entity
#     for prop in entity.get_properties():
#         for value in prop[entity]:
#             if hasattr(value, "name"):  # Ensure value is another entity
#                 nodes.add(value.name)
#                 edges.append((entity.name, prop.python_name, value.name))
    
#     # Add metadata for specific node types
#     if "Patient" in [cls.name for cls in entity.is_a]:
#         metadata[entity.name] = {
#             "age": getattr(entity, "hasAge", ["NA"])[0],
#             "gender": getattr(entity, "hasGender", ["NA"])[0],
#             "disease_status": getattr(entity, "hasDiseaseStatus", ["NA"])[0],
#         }
#     elif "Sample" in [cls.name for cls in entity.is_a]:
#         metadata[entity.name] = {"num_genes": len(getattr(entity, "hasGeneExpression", []))}

# # Convert nodes and edges to DataFrame
# edges_df = pd.DataFrame(edges, columns=["Source", "Relation", "Target"])
# nodes_df = pd.DataFrame(list(nodes), columns=["Node"])

# # Display extracted data
# print("### Nodes ###")
# print(nodes_df.head())
# print("\n### Edges ###")
# print(edges_df.head())
# print("\n### Metadata ###")
# print(pd.DataFrame.from_dict(metadata, orient="index").head())


In [173]:
from owlready2 import get_ontology
import pandas as pd

# Load ontology
ontology_path = "enriched_ppio_ontology_final_54_small.owl"
ontology = get_ontology(ontology_path).load()

# Relationships of interest
relations_of_interest = ["hasSample", "hasGeneExpression", "isAssociatedWithPathway", "isAssociatedWith"]

# Initialize storage
nodes = set()
edges = []
edge_metadata = {}  # Store expression values
metadata = {}

# Helper function to safely get attribute values
def safe_getattr(entity, attr, default="NA"):
    """Returns first value if available, otherwise returns default"""
    values = getattr(entity, attr, [])
    return values[0].name if values and hasattr(values[0], "name") else (values[0] if values else default)

# Process individuals in ontology
for entity in ontology.individuals():
    entity_name = entity.name
    nodes.add(entity_name)

    # Extract edges
    for prop in entity.get_properties():
        if prop.python_name in relations_of_interest:
            for value in prop[entity]:
                if hasattr(value, "name"):  
                    nodes.add(value.name)
                    edges.append((entity_name, prop.python_name, value.name))

    # Identify entity class type
    entity_classes = [cls.name for cls in entity.is_a]

    # Store metadata for Patients
    if "Patient" in entity_classes:
        metadata[entity_name] = {
            "type": "Patient",
            "age": safe_getattr(entity, "hasAge"),
            "gender": safe_getattr(entity, "hasGender"),
            "disease_status": safe_getattr(entity, "hasDiseaseStatus"),
            "group_id": safe_getattr(entity, "hasGroupId"),
            "site_of_infection": safe_getattr(entity, "hasSiteOfInfection"),
            "severity": safe_getattr(entity, "hasSeverity"),
        }

    # Store metadata for Samples
    elif "Sample" in entity_classes:
        metadata[entity_name] = {
            "type": "Sample",
            "num_genes": len(getattr(entity, "hasGeneExpression", [])),
            "source": safe_getattr(entity, "isSourcedFrom"),
            "tissue": safe_getattr(entity, "hasTissue"),
            "neutrophil_proportion": safe_getattr(entity, "hasNeutrophilProportion"),
        }

    # Store metadata for Genes
    elif "Gene" in entity_classes:
        metadata[entity_name] = {
            "type": "Gene",
            "entrez_id": safe_getattr(entity, "hasEntrezId"),
            "pathway": safe_getattr(entity, "isAssociatedWithPathway"),
        }

# Extract annotated edges (Gene-Sample expression values) using get_triples()
for subj, pred, obj in ontology.get_triples():
    subj_name = subj.name if hasattr(subj, "name") else None
    pred_name = pred.name if hasattr(pred, "name") else None
    obj_name = obj.name if hasattr(obj, "name") else None

    # Check if it's an expression value annotation
    if pred_name == "hasExpressionValue":
        if subj_name and obj_name:
            expression_value = float(obj) if isinstance(obj, (int, float)) else "NA"
            edges.append((subj_name, "hasGeneExpression", obj_name))
            edge_metadata[(subj_name, obj_name)] = expression_value  

# Convert to DataFrames
edges_df = pd.DataFrame(edges, columns=["Source", "Relation", "Target"])
nodes_df = pd.DataFrame(list(nodes), columns=["Node"])

# Add expression values to edges DataFrame
edges_df["ExpressionValue"] = edges_df.apply(lambda row: edge_metadata.get((row["Source"], row["Target"]), "NA"), axis=1)

# Convert metadata dictionary to DataFrame
metadata_df = pd.DataFrame.from_dict(metadata, orient="index")
metadata_df.fillna("NA", inplace=True)
metadata_df = metadata_df.astype(str)  # Ensure consistent data format



## Create Node Features

In [176]:
from owlready2 import get_ontology
import pandas as pd

# Load ontology
ontology_path = "enriched_ppio_ontology_final_54_small.owl"
ontology = get_ontology(ontology_path).load()

# Storage
nodes = set()
edges = []
edge_metadata = {}  # Store expression values
metadata = {}

# Helper function to safely get attribute values
def safe_getattr(entity, attr, default="NA"):
    values = getattr(entity, attr, [])
    return values[0].name if values and hasattr(values[0], "name") else (values[0] if values else default)

# Process individuals
for entity in ontology.individuals():
    entity_name = entity.name
    nodes.add(entity_name)

    # Extract edges
    for prop in entity.get_properties():
        for value in prop[entity]:
            if hasattr(value, "name"):
                nodes.add(value.name)
                edges.append((entity_name, prop.python_name, value.name))

    # Identify entity type and metadata
    entity_classes = [cls.name for cls in entity.is_a]
    
    if "Patient" in entity_classes:
        metadata[entity_name] = {
            "type": "Patient",
            "age": safe_getattr(entity, "hasAge"),
            "gender": safe_getattr(entity, "hasGender"),
            "disease_status": safe_getattr(entity, "hasDiseaseStatus"),
            "group_id": safe_getattr(entity, "hasGroupId"),
            "site_of_infection": safe_getattr(entity, "hasSiteOfInfection"),
            "severity": safe_getattr(entity, "hasSeverity"),
        }
    elif "Sample" in entity_classes:
        metadata[entity_name] = {
            "type": "Sample",
            "num_genes": len(getattr(entity, "hasGeneExpression", [])),
            "tissue": safe_getattr(entity, "hasTissue"),
            "neutrophil_proportion": safe_getattr(entity, "hasNeutrophilProportion"),
        }
    elif "Gene" in entity_classes:
        metadata[entity_name] = {
            "type": "Gene",
            "entrez_id": safe_getattr(entity, "hasEntrezId"),
            "pathway": safe_getattr(entity, "isAssociatedWithPathway"),
        }

# Extract annotated edges with expression values
for subj, pred, obj in ontology.get_triples():
    if hasattr(pred, "name") and pred.name == "hasExpressionValue":
        source = subj.name if hasattr(subj, "name") else None
        target = obj.name if hasattr(obj, "name") else None
        value = getattr(obj, "value", None)  # Get numerical expression value
        
        if source and target and value:
            edge_metadata[(source, target)] = float(value) if isinstance(value, (int, float)) else "NA"

# Convert to DataFrames
edges_df = pd.DataFrame(edges, columns=["Source", "Relation", "Target"])

# Assign expression values
edges_df["ExpressionValue"] = edges_df.apply(lambda row: edge_metadata.get((row["Source"], row["Target"]), "NA"), axis=1)

# Convert metadata to DataFrame
metadata_df = pd.DataFrame.from_dict(metadata, orient="index").fillna("NA").astype(str)


In [177]:
# Display extracted data
print("### Nodes ###")
print(nodes_df.head(10))
print("\n### Edges ###")
print(edges_df.head(10))
print("\n### Metadata ###")
print(metadata_df.head(10))

### Nodes ###
              Node
0       Gene_54458
1      Gene_728518
2       Gene_64432
3      Gene_170960
4      Gene_115548
5        Gene_5504
6       GO_0016605
7     Patient_S_46
8  Protein_PRKAR1A
9   Protein_GOLGA3

### Edges ###
              Source           Relation             Target ExpressionValue
0       Patient_HC_1           isPartOf           GSE54514              NA
1       Patient_HC_1          hasSample  Sample_GSM1317896              NA
2       Patient_HC_1          hasSample  Sample_GSM1318041              NA
3  Sample_GSM1317896  hasGeneExpression         Gene_60496              NA
4  Sample_GSM1317896  hasGeneExpression            Gene_18              NA
5  Sample_GSM1317896  hasGeneExpression         Gene_10347              NA
6  Sample_GSM1317896  hasGeneExpression           Gene_215              NA
7  Sample_GSM1317896  hasGeneExpression            Gene_23              NA
8  Sample_GSM1317896  hasGeneExpression            Gene_26              NA
9  Sample_GS

## Define GCN for Link Prediction

In [140]:
# Define the GCN model
class GCNLinkPredictor(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCNLinkPredictor, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        return x

# Initialize model, optimizer, and loss function
input_dim = node_features_tensor.size(1)
model = GCNLinkPredictor(input_dim=input_dim, hidden_dim=64, output_dim=32)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
loss_fn = torch.nn.BCEWithLogitsLoss()


## Train the Model

In [141]:
from sklearn.model_selection import train_test_split

# Split edges into train and test sets
edges_train, edges_test = train_test_split(edge_index.t().tolist(), test_size=0.2, random_state=42)

# Convert to tensors
edges_train = torch.tensor(edges_train, dtype=torch.long).t()
edges_test = torch.tensor(edges_test, dtype=torch.long).t()

# Generate negative samples
def generate_negative_samples(edge_index, num_nodes, num_neg_samples):
    neg_samples = negative_sampling(
        edge_index=edge_index, num_nodes=num_nodes, num_neg_samples=num_neg_samples
    )
    return neg_samples

# Training loop
for epoch in range(100):
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    embeddings = model(node_features_tensor, edges_train)
    
    # Positive edges
    pos_edge_scores = (embeddings[edges_train[0]] * embeddings[edges_train[1]]).sum(dim=1)
    
    # Negative edges
    neg_edges = generate_negative_samples(edges_train, len(nodes), edges_train.size(1))
    neg_edge_scores = (embeddings[neg_edges[0]] * embeddings[neg_edges[1]]).sum(dim=1)
    
    # Compute loss
    pos_labels = torch.ones(pos_edge_scores.size(0))
    neg_labels = torch.zeros(neg_edge_scores.size(0))
    loss = loss_fn(torch.cat([pos_edge_scores, neg_edge_scores]), torch.cat([pos_labels, neg_labels]))
    
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")


Epoch 1, Loss: 2075.3838
Epoch 2, Loss: 868.5792
Epoch 3, Loss: 300.2424
Epoch 4, Loss: 156.5396
Epoch 5, Loss: 192.4181
Epoch 6, Loss: 235.3768
Epoch 7, Loss: 230.8152
Epoch 8, Loss: 191.8416
Epoch 9, Loss: 152.7773
Epoch 10, Loss: 124.4660
Epoch 11, Loss: 106.1344
Epoch 12, Loss: 92.8360
Epoch 13, Loss: 82.2695
Epoch 14, Loss: 73.5834
Epoch 15, Loss: 64.2566
Epoch 16, Loss: 53.9726
Epoch 17, Loss: 43.8109
Epoch 18, Loss: 34.7930
Epoch 19, Loss: 27.3244
Epoch 20, Loss: 22.1782
Epoch 21, Loss: 19.0790
Epoch 22, Loss: 17.6460
Epoch 23, Loss: 17.7682
Epoch 24, Loss: 18.2199
Epoch 25, Loss: 18.7809
Epoch 26, Loss: 18.8495
Epoch 27, Loss: 18.5286
Epoch 28, Loss: 17.1012
Epoch 29, Loss: 15.3378
Epoch 30, Loss: 13.1323
Epoch 31, Loss: 10.8185
Epoch 32, Loss: 8.6821
Epoch 33, Loss: 6.8919
Epoch 34, Loss: 5.5021
Epoch 35, Loss: 4.4623
Epoch 36, Loss: 3.8033
Epoch 37, Loss: 3.4712
Epoch 38, Loss: 3.3428
Epoch 39, Loss: 3.3545
Epoch 40, Loss: 3.4015
Epoch 41, Loss: 3.4280
Epoch 42, Loss: 3.4166


## Evaluate the Model

In [143]:
from sklearn.metrics import roc_auc_score
import torch.nn.functional as F

# Evaluate link prediction performance
model.eval()
with torch.no_grad():
    test_embeddings = model(node_features_tensor, edge_index)
    
    # Test positive edges
    test_pos_scores = (test_embeddings[edges_test[0]] * test_embeddings[edges_test[1]]).sum(dim=1)
    
    # Test negative edges
    test_neg_edges = generate_negative_samples(edges_test, len(nodes), edges_test.size(1))
    test_neg_scores = (test_embeddings[test_neg_edges[0]] * test_embeddings[test_neg_edges[1]]).sum(dim=1)
    
    # Compute probability scores
    y_true = torch.cat([torch.ones(test_pos_scores.size(0)), torch.zeros(test_neg_scores.size(0))])
    y_pred = torch.cat([test_pos_scores, test_neg_scores]).sigmoid()  # Apply sigmoid

    # Compute AUC-ROC
    auc_roc = roc_auc_score(y_true.cpu().numpy(), y_pred.cpu().numpy())
    print(f"AUC-ROC Score: {auc_roc:.4f}")

    # Compute Mean Reciprocal Rank (MRR)
    ranks = torch.argsort(torch.argsort(-y_pred))
    mrr = (1 / (1 + ranks)).mean().item()
    print(f"Mean Reciprocal Rank (MRR): {mrr:.4f}")

    # Compute Hits@K
    def compute_hits_at_k(scores, k):
        return (scores.topk(k)[1] < k).float().mean().item()

    hits_at_1 = compute_hits_at_k(y_pred, 1)
    hits_at_3 = compute_hits_at_k(y_pred, 3)
    hits_at_10 = compute_hits_at_k(y_pred, 10)

    print(f"Hits@1: {hits_at_1:.4f}")
    print(f"Hits@3: {hits_at_3:.4f}")
    print(f"Hits@10: {hits_at_10:.4f}")


AUC-ROC Score: 0.6103
Mean Reciprocal Rank (MRR): 0.0001
Hits@1: 0.0000
Hits@3: 0.0000
Hits@10: 0.0000


In [178]:
import rdflib
import random

def summarize_owl_file(input_owl, output_owl, sample_ratio=0.3, min_triples=50):
    """
    Summarizes an OWL file by randomly sampling a subset of triples while preserving key entities and relationships.
    
    Parameters:
    - input_owl (str): Path to the input OWL file.
    - output_owl (str): Path to save the summarized OWL file.
    - sample_ratio (float): Percentage of triples to retain (default 30%).
    - min_triples (int): Minimum number of triples to retain (default 50).
    """
    # Load the OWL file
    graph = rdflib.Graph()
    graph.parse(input_owl, format="xml")

    # Extract all triples
    all_triples = list(graph)

    # Determine number of triples to retain
    num_triples_to_keep = max(int(len(all_triples) * sample_ratio), min_triples)
    
    # Randomly sample triples
    sampled_triples = random.sample(all_triples, min(num_triples_to_keep, len(all_triples)))

    # Create a new graph and add the sampled triples
    summarized_graph = rdflib.Graph()
    for triple in sampled_triples:
        summarized_graph.add(triple)

    # Save the summarized OWL file
    summarized_graph.serialize(destination=output_owl, format="xml")

    print(f"Original Triples: {len(all_triples)}")
    print(f"Summarized Triples: {len(sampled_triples)}")
    print(f"Summarized OWL file saved as: {output_owl}")

# Example usage
input_owl_file = "enriched_ppio_ontology_final_54_small.owl"  # Replace with your actual OWL file
output_owl_file = "enriched_ppio_ontology_final_54_summarized.owl"

summarize_owl_file(input_owl_file, output_owl_file, sample_ratio=0.3, min_triples=50)


Original Triples: 2906769
Summarized Triples: 872030
Summarized OWL file saved as: enriched_ppio_ontology_final_54_summarized.owl


In [1]:
import rdflib
import torch
import torch.nn as nn
import torch.optim as optim
import dgl
from dgl.nn import RelGraphConv
from collections import defaultdict
import numpy as np
import random

# Step 1: Load OWL File and Extract Triples
def load_graph_from_owl(file_path, relations_of_interest):
    g = rdflib.Graph()
    g.parse(file_path, format="xml")
    
    triples = []
    nodes = set()
    
    for subj, pred, obj in g:
        pred_short = str(pred).split("/")[-1]  # Extract relation name
        if pred_short in relations_of_interest:
            triples.append((str(subj), pred_short, str(obj)))
            nodes.add(str(subj))
            nodes.add(str(obj))
    
    return triples, list(nodes)

# Step 2: Convert to Numeric ID Mappings
def build_mappings(triples, nodes):
    node2id = {n: i for i, n in enumerate(nodes)}
    relation2id = {r: i for i, r in enumerate(set([t[1] for t in triples]))}
    
    edge_list = [(node2id[s], node2id[o], relation2id[r]) for s, r, o in triples]
    return node2id, relation2id, edge_list

# Step 3: Create DGL Graph
def build_dgl_graph(edge_list, num_nodes, num_relations):
    src, dst, rel = zip(*edge_list)
    g = dgl.graph((torch.tensor(src), torch.tensor(dst)), num_nodes=num_nodes)
    g.edata['etype'] = torch.tensor(rel)
    return g

# Step 4: Define R-GCN Model
class RGCN(nn.Module):
    def __init__(self, num_nodes, num_relations, hidden_dim, num_layers):
        super(RGCN, self).__init__()
        self.embedding = nn.Embedding(num_nodes, hidden_dim)
        self.layers = nn.ModuleList(
            [RelGraphConv(hidden_dim, hidden_dim, num_relations, "basis", num_bases=5) for _ in range(num_layers)]
        )
    
    def forward(self, g):
        h = self.embedding.weight
        for layer in self.layers:
            h = layer(g, h, g.edata['etype'])
        return h

# Step 5: Train Model
def train_model(graph, num_nodes, num_relations, edge_list):
    model = RGCN(num_nodes, num_relations, hidden_dim=16, num_layers=2)
    optimizer = optim.Adam(model.parameters(), lr=0.01)
    loss_fn = nn.MSELoss()
    
    for epoch in range(50):
        model.train()
        optimizer.zero_grad()
        embeddings = model(graph)
        loss = loss_fn(embeddings[torch.tensor([e[0] for e in edge_list])], embeddings[torch.tensor([e[1] for e in edge_list])])
        loss.backward()
        optimizer.step()
        print(f"Epoch {epoch+1}, Loss: {loss.item()}")
    
    return model, embeddings

# Load and Prepare Data
owl_file = "enriched_ppio_ontology_final_54_small.owl.owl"
relations_of_interest = ["hasSample", "hasGeneExpression", "isAssociatedWithPathway", "isAssociatedWith"]
triples, nodes = load_graph_from_owl(owl_file, relations_of_interest)
node2id, relation2id, edge_list = build_mappings(triples, nodes)

FileNotFoundError: Cannot find DGL C++ graphbolt library at /opt/homebrew/Cellar/jupyterlab/4.3.1_1/libexec/lib/python3.12/site-packages/dgl/graphbolt/libgraphbolt_pytorch_2.5.1.dylib

In [9]:
!git clone --recursive https://github.com/dmlc/dgl.git


Cloning into 'dgl'...
remote: Enumerating objects: 57222, done.
remote: Counting objects: 100% (6576/6576), done. Counting objects: 100% (6576/6576)
remote: Compressing objects: 100% (779/779), done. Compressing objects:  20% (156/779)remote: Compressing objects:  25% (195/779)remote: Compressing objects:  27% (211/779)
Receiving objects:  98% (56078/57222), 25.22 MiB | 12.17 MiB/s57222)Receiving objects:   2% (1145/57222)Receiving objects:   5% (2862/57222)Receiving objects:   7% (4006/57222)Receiving objects:   8% (4578/57222)Receiving objects:  12% (6867/57222)Receiving objects:  13% (7439/57222), 5.19 MiB | 9.09 MiB/sReceiving objects:  15% (8584/57222), 5.19 MiB | 9.09 MiB/sReceiving objects:  17% (9728/57222), 5.19 MiB | 9.09 MiB/sReceiving objects:  18% (10300/57222), 5.19 MiB | 9.09 MiB/sReceiving objects:  22% (12708/57222), 5.19 MiB | 9.09 MiB/sReceiving objects:  25% (14306/57222), 11.84 MiB | 11.04 MiB/sReceiving objects:  35% (20028/57222), 11.84 MiB | 11.04 MiB/sReceiving

CMake Warning:
  Ignoring extra path from command line:

   ".."


CMake Error: The source directory "/Users/pratistha99/Desktop/BDMA/CS/subjects" does not appear to contain CMakeLists.txt.
Specify --help for usage, or press the help button on the CMake GUI.


In [ ]:
# Build Graph and Train Model
graph = build_dgl_graph(edge_list, num_nodes=len(nodes), num_relations=len(relation2id))
model, embeddings = train_model(graph, len(nodes), len(relation2id), edge_list)